# DataSource

Node definition for defining access and structure of data sources. The base class for data sources is `podpac.data.DataSource`.

The child class must implement the method `get_data`.
This method defines how to select data from the source given input PODPAC Coordinates.

The child class often implements the method `get_native_coordinates`.
This method defines defines how to select the native coordinates from the source and returns PODPAC Coordinates.

## Array Example

In [1]:
from podpac.data import DataSource
from podpac import Coordinates
import traitlets as tl     # traitlets is used to enforce types
import numpy as np

class MyDataType(DataSource):
    
    source = tl.Instance(np.ndarray)   # input "source" to this data type must be a numpy ndarray
                                       # DataSource includes input "native_coordinates" and enforces Coordinates type

    def get_data(self, coordinates, coordinates_index):
        # this will run when the node is evaluated
        
        # use the index of the coordinates to select data from the "source"
        idx = coordinates_index
        data = self.source[idx]
        
        # Create a UnitsDataArray from coordinates and data from "source"
        uda = self.create_output_array(coordinates, data=data)
        
        return uda

D:\podpac-1.1.0\miniconda\envs\podpac\lib\site-packages\xarray\core\merge.py:17: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  PANDAS_TYPES = (pd.Series, pd.DataFrame, pd.Panel)


In [2]:
# make a node from your DataSource
data = np.random.rand(5, 5)   # random 5x5 grid of data
native_coordinates = Coordinates([np.linspace(0, 5, 5), np.linspace(0, 5, 5)], dims=['lat', 'lon'])

node = MyDataType(source=data, native_coordinates=native_coordinates)
node

MyDataType DataSource
	source: 
[[0.4337754  0.99017508 0.49285038 0.58936728 0.54340694]
 [0.76951822 0.99811427 0.06203534 0.4966574  0.48687567]
 [0.94014489 0.49938377 0.2248681  0.4554301  0.03547552]
 [0.80863856 0.29523344 0.67475007 0.95272911 0.91748385]
 [0.88202562 0.92128363 0.07143294 0.39035306 0.85822818]]
	native_coordinates: 
		lat: ArrayCoordinates1d(lat): Bounds[0.0, 5.0], N[5], ctype['midpoint']
		lon: ArrayCoordinates1d(lon): Bounds[0.0, 5.0], N[5], ctype['midpoint']
	interpolation: nearest

In [3]:
# evaluate the node at coordinates in the top left of the grid
coords = Coordinates([[0, 1, 2], [0, 1, 2]], dims=['lat', 'lon'])
output = node.eval(coords)
output

<xarray.UnitsDataArray (lat: 3, lon: 3)>
array([[0.433775, 0.990175, 0.49285 ],
       [0.769518, 0.998114, 0.062035],
       [0.940145, 0.499384, 0.224868]])
Coordinates:
  * lat      (lat) float64 0.0 1.0 2.0
  * lon      (lon) float64 0.0 1.0 2.0
Attributes:
    layer_style:  <podpac.core.style.Style object at 0x000001C1CC4396D8>
    crs:          EPSG:4326

## DataFrame Example

In [4]:
from podpac.data import DataSource
from podpac import Coordinates

import traitlets as tl     # traitlets is used to enforce types
import numpy as np
import pandas as pd

class OtherType(DataSource):
    
    source = tl.Instance(pd.DataFrame)   # input "source" to this data type must be a pandas DataFrame
    lat_key = tl.Unicode()               # where to find "lat" data in this dataframe
    lon_key = tl.Unicode()               # where to find "lon" in this dataframe
    data_key = tl.Unicode()              # where to find specific data of interest in this dataframe
    
    def get_native_coordinates(self):
        # this will return native coordinates as a podpac.Coordinates class
        
        lat = self.source[self.lat_key]
        lon = self.source[self.lon_key]
        
        # since this is tabular dat, our coordinates are point data, or "stacked"
        # in podpac we use the "_" to refer to stacked coordinates
        return Coordinates([(lat, lon)], dims=['lat_lon'])
    
    
    def get_data(self, coordinates, coordinates_index):
        # this will run when the node is evaluated
        
        # use the index of the coordinates to select data from the "source"
        idx = coordinates_index
        data = self.source[self.data_key].iloc[idx]
        
        # Create a UnitsDataArray from coordinates and data from "source"
        uda = self.create_output_array(coordinates, data=data)
        
        return uda

In [5]:
# make a node from your DataSource
data = pd.DataFrame(data={
    'values': np.random.rand(5),
    'lon_val': np.linspace(0, 5, 5),
    'lat_val': np.linspace(0, 5, 5)
})

node = OtherType(source=data,
                 lat_key='lat_val',
                 lon_key='lon_val',
                 data_key='values'
                )
node

OtherType DataSource
	source: 
     values  lon_val  lat_val
0  0.155119     0.00     0.00
1  0.747241     1.25     1.25
2  0.400586     2.50     2.50
3  0.331538     3.75     3.75
4  0.910010     5.00     5.00
	interpolation: nearest

In [6]:
# evaluate the node at coordinates in the top left of the grid
coords = Coordinates([([0, 1, 2], [0, 1, 2])], dims=['lat_lon'])
output = node.eval(coords)
output

<xarray.UnitsDataArray (lat_lon: 3)>
array([0.155119, 0.747241, 0.400586])
Coordinates:
  * lat_lon  (lat_lon) MultiIndex
  - lat      (lat_lon) float64 0.0 1.0 2.0
  - lon      (lat_lon) float64 0.0 1.0 2.0
Attributes:
    layer_style:  <podpac.core.style.Style object at 0x000001C1CC4B1240>
    crs:          EPSG:4326